In [31]:
# # Settings
# import os
# from os.path import join

# # TRACKING
# W = 5  # stride of the grid for sampling new tracks
# L = 15  # number of frames for each complete trajectory
# of_winsize = 20  # optical flow window size

# # Sh-Tomasi threshold on cornerness response function values
# # 0.001 as in the paper detects too many corners with some corners in homogeneous regions
# # maybe because here we use only 1 spatial scale and no smoothing, so we need a stricter threshold
# corner_quality_level = 0.01
# static_displacement_thresh = 5
# max_single_displacement = 0.7


# # TUBES AND DESCRIPTORS
# N = 32  # side of the square around the tracked point (1-frame section of the tube)
# N2 = int(N / 2)
# n_sigma = 2  # number of tube grid cells by x and y
# n_tau = 3  # number of tube grid cells in time
# bins = 8  # number of bins for all the descriptors except HoF, which would be bins + 1
# len_descriptor = L * 2 + 3 * bins * n_sigma * n_sigma * n_tau + (bins + 1) * n_sigma * n_sigma * n_tau
# bin_angles = [int(i * 360 / bins) for i in range(bins + 1)]
# small_flow_magnitude = 1


# # PCA AND GMM
# pca_num_components = 32
# gmm_n_components = 3


# # PATHS
# video_descriptors_path = join('out', 'video_descriptors')
# data_dir = 'data_avi_hd'


In [75]:
# First half, create fisher vectors for classification

%load_ext autoreload
%autoreload 2
import os
from os.path import join
import numpy as np
import sklearn.decomposition as decomp
from sklearn.mixture import GaussianMixture as GMM
from sklearn.svm import LinearSVC

from video_representation import VideoRepresentation
from transforms import *
from settings import *
from visualize import *
from tqdm import tqdm
from yellowbrick.cluster import SilhouetteVisualizer

already_computed_descriptors=False

try:
    os.makedirs(video_descriptors_path)
except Exception as error:
    pass

train_videos = []
test_videos = []

# COMPUTE DESCRIPTORS
if not already_computed_descriptors:
    for id, directory in enumerate(next(os.walk(data_dir))[1]):
        if id == 3: # debugging, skip ollie directory
            continue
        directory_path = join(data_dir, directory)
        print(f'\n________EXTRACTING DESCRIPTORS FROM {directory_path}')
        for filename in tqdm(os.listdir(directory_path)):
            filepath = join(directory_path, filename)
            # check if file already exits
            descriptors_txt_path = os.path.join(video_descriptors_path, f'{filename.split(".")[0]}-descriptors.txt')
            descriptor_found = False
            if os.path.isfile(descriptors_txt_path):
                print('descriptor file already exists: ', descriptors_txt_path)
                descriptor_found = True
            if '.avi' in filename and os.path.isfile(filepath) and not descriptor_found:
                # task 1
                trajectories_list = trajectories_from_video(filepath)
                # task 2
                # saves descriptors to disk
                descriptors_from_trajectories(trajectories_list, filename)

# TRAIN
train_lines = []
with open(join(data_dir, 'train.txt'), 'r') as train_f:
    train_lines = train_f.readlines()
for l in train_lines:
    # Handle skateboard trick file paths
    label = 0
    filepath = l.replace('\n', '')
    if 'Kickflip' in l:
        label = 1
    # filepath, label = l.split() junp.avi 1
    
    descriptor_path = join(video_descriptors_path,
                        f'{filepath.split("/")[1].replace(".avi", "-descriptors.txt")}')
    video_representation = VideoRepresentation(filepath, np.loadtxt(descriptor_path), label)
    train_videos.append(video_representation)

all_train_descriptors = np.concatenate([v.descriptors for v in train_videos], axis=0)
print(f'total number of train descriptors: {all_train_descriptors.shape[0]}')
print(f'length of each train descriptor: {all_train_descriptors.shape[1]}')

# init and fit the pca
pca = decomp.PCA(pca_num_components)
pca = pca.fit(all_train_descriptors)

# transform descriptors of each video
for v in train_videos:
    v.pca_descriptors = pca.transform(v.descriptors)

# concatenate the pca-transformed descriptors, to not transform the whole data one extra time
all_train_descriptors = np.concatenate([v.pca_descriptors for v in train_videos], axis=0)
print(f'length each train descriptor after pca: {all_train_descriptors.shape[1]}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
total number of train descriptors: 1223186
length of each train descriptor: 426
length each train descriptor after pca: 64


In [76]:
%load_ext autoreload
%autoreload 2
from settings import *
print(pca_num_components)
print(gmm_n_components)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
64
3


In [77]:
# learn GMM model
gmm = GMM(n_components=gmm_n_components, covariance_type='diag')
gmm.fit(all_train_descriptors)

# Check gmm isn't overfitting
print(f'gmm score: {gmm.score(all_train_descriptors)}')

gmm score: 41.34493344000595


In [78]:
# compute fisher vectors for each train video
for v in train_videos:
    v.fisher_vector = fisher_from_descriptors(v.pca_descriptors, gmm)
print('calculated Fisher vectors')

calculated Fisher vectors


In [79]:
# Standard model

# initialize and fit a linear SVM
model = LinearSVC()
model.fit(X=[v.fisher_vector for v in train_videos], y=[v.label for v in train_videos])
print('fitted linear SVM')

fitted linear SVM


/Users/hugoderohanwillner/anaconda3/envs/MSc_Vision/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/hugoderohanwillner/anaconda3/envs/MSc_Vision/lib/python3.8/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [80]:
# Check training accuracy
training_accuracy = model.score(X=[v.fisher_vector for v in train_videos], y=[v.label for v in train_videos])
print(f'training accuracy: {training_accuracy}')

# Check individual labels
prediction = model.predict(X=[v.fisher_vector for v in train_videos])
for i, v in enumerate(train_videos):
    v.predicted_label = prediction[i]
print('prediction by video: index, true label, predicted label, path\n')
for i, v in enumerate(train_videos):
    print(f'{i}    gt: {v.label}    pred: {v.predicted_label}   {v.filepath}')

training accuracy: 1.0
prediction by video: index, true label, predicted label, path

0    gt: 1    pred: 1   Kickflip/Kickflip69.avi
1    gt: 1    pred: 1   Kickflip/Kickflip33.avi
2    gt: 1    pred: 1   Kickflip/Kickflip100.avi
3    gt: 1    pred: 1   Kickflip/Kickflip58.avi
4    gt: 1    pred: 1   Kickflip/Kickflip68.avi
5    gt: 1    pred: 1   Kickflip/Kickflip40.avi
6    gt: 1    pred: 1   Kickflip/Kickflip109.avi
7    gt: 1    pred: 1   Kickflip/Kickflip85.avi
8    gt: 1    pred: 1   Kickflip/Kickflip14.avi
9    gt: 1    pred: 1   Kickflip/Kickflip26.avi
10    gt: 1    pred: 1   Kickflip/Kickflip5.avi
11    gt: 1    pred: 1   Kickflip/Kickflip44.avi
12    gt: 1    pred: 1   Kickflip/Kickflip73.avi
13    gt: 1    pred: 1   Kickflip/Kickflip13.avi
14    gt: 1    pred: 1   Kickflip/Kickflip49.avi
15    gt: 1    pred: 1   Kickflip/Kickflip57.avi
16    gt: 1    pred: 1   Kickflip/Kickflip2.avi
17    gt: 1    pred: 1   Kickflip/Kickflip83.avi
18    gt: 1    pred: 1   Kickflip/Kickflip

In [81]:
# TEST

print('model: ', model )

test_lines = []
with open(join(data_dir, 'test.txt'), 'r') as test_f:
    test_lines = test_f.readlines()
for l in test_lines:
    # Handle skateboard trick file paths
    label = 0
    filepath = l.replace('\n', '')
    if 'Kickflip' in l:
        label = 1
    # filepath, label = l.split() junp.avi 1
    descriptor_path = join(video_descriptors_path,
                        f'{filepath.split("/")[1].replace(".avi", "-descriptors.txt")}')
    video_representation = VideoRepresentation(filepath, np.loadtxt(descriptor_path), label)
    test_videos.append(video_representation)

# reduce dimension of all test descriptors using pca fitted on train data
for v in test_videos:
    v.pca_descriptors = pca.transform(v.descriptors)
print('reduced dimensions of the test data')

# calculate a fisher vector for each test video based on the gmm model fit on the train data
for v in test_videos:
    v.fisher_vector = fisher_from_descriptors(v.pca_descriptors, gmm)
print('calculated Fisher vectors on the test data')

# predict the labels of the test videos
accuracy = model.score(X=[v.fisher_vector for v in test_videos], y=[v.label for v in test_videos])
print(f'accuracy: {accuracy}')
prediction = svm.predict(X=[v.fisher_vector for v in test_videos])
for i, v in enumerate(test_videos):
    v.predicted_label = prediction[i]
print('prediction by video: index, true label, predicted label, path\n')
for i, v in enumerate(test_videos):
    print(f'{i}    gt: {v.label}    pred: {v.predicted_label}   {v.filepath}')





    # to test trajectories on a single video
    # trajectories_from_video('data_avi_hd/Ollie/Ollie20.avi', vis_flow=False, vis_trajectories=True)

model:  LinearSVC()
reduced dimensions of the test data
calculated Fisher vectors on the test data
accuracy: 0.6984126984126984
prediction by video: index, true label, predicted label, path

0    gt: 1    pred: 0   Kickflip/Kickflip111.avi
1    gt: 1    pred: 1   Kickflip/Kickflip95.avi
2    gt: 1    pred: 1   Kickflip/Kickflip51.avi
3    gt: 1    pred: 0   Kickflip/Kickflip20.avi
4    gt: 1    pred: 1   Kickflip/Kickflip80.avi
5    gt: 1    pred: 1   Kickflip/Kickflip70.avi
6    gt: 1    pred: 1   Kickflip/Kickflip104.avi
7    gt: 1    pred: 1   Kickflip/Kickflip65.avi
8    gt: 1    pred: 1   Kickflip/Kickflip32.avi
9    gt: 1    pred: 1   Kickflip/Kickflip64.avi
10    gt: 1    pred: 1   Kickflip/Kickflip87.avi
11    gt: 1    pred: 1   Kickflip/Kickflip54.avi
12    gt: 1    pred: 0   Kickflip/Kickflip45.avi
13    gt: 1    pred: 1   Kickflip/Kickflip60.avi
14    gt: 1    pred: 1   Kickflip/Kickflip23.avi
15    gt: 1    pred: 1   Kickflip/Kickflip63.avi
16    gt: 1    pred: 1   Kickflip

In [68]:
all = 'Kickflip/Kickflip0.avi Kickflip/Kickflip1.avi Kickflip/Kickflip2.avi Kickflip/Kickflip3.avi Kickflip/Kickflip4.avi Kickflip/Kickflip5.avi Kickflip/Kickflip6.avi Kickflip/Kickflip7.avi Kickflip/Kickflip8.avi Kickflip/Kickflip9.avi Kickflip/Kickflip10.avi Kickflip/Kickflip11.avi Kickflip/Kickflip12.avi Kickflip/Kickflip13.avi Kickflip/Kickflip14.avi Kickflip/Kickflip15.avi Kickflip/Kickflip16.avi Kickflip/Kickflip17.avi Kickflip/Kickflip18.avi Kickflip/Kickflip19.avi Kickflip/Kickflip20.avi Kickflip/Kickflip21.avi Kickflip/Kickflip22.avi Kickflip/Kickflip23.avi Kickflip/Kickflip24.avi Kickflip/Kickflip25.avi Kickflip/Kickflip26.avi Kickflip/Kickflip27.avi Kickflip/Kickflip28.avi Kickflip/Kickflip29.avi Kickflip/Kickflip30.avi Kickflip/Kickflip32.avi Kickflip/Kickflip33.avi Kickflip/Kickflip34.avi Kickflip/Kickflip35.avi Kickflip/Kickflip36.avi Kickflip/Kickflip37.avi Kickflip/Kickflip38.avi Kickflip/Kickflip39.avi Kickflip/Kickflip40.avi Kickflip/Kickflip41.avi Kickflip/Kickflip42.avi Kickflip/Kickflip43.avi Kickflip/Kickflip44.avi Kickflip/Kickflip45.avi Kickflip/Kickflip46.avi Kickflip/Kickflip47.avi Kickflip/Kickflip48.avi Kickflip/Kickflip49.avi Kickflip/Kickflip50.avi Kickflip/Kickflip51.avi Kickflip/Kickflip52.avi Kickflip/Kickflip53.avi Kickflip/Kickflip54.avi Kickflip/Kickflip55.avi Kickflip/Kickflip56.avi Kickflip/Kickflip57.avi Kickflip/Kickflip58.avi Kickflip/Kickflip59.avi Kickflip/Kickflip60.avi Kickflip/Kickflip61.avi Kickflip/Kickflip62.avi Kickflip/Kickflip63.avi Kickflip/Kickflip64.avi Kickflip/Kickflip65.avi Kickflip/Kickflip66.avi Kickflip/Kickflip67.avi Kickflip/Kickflip68.avi Kickflip/Kickflip69.avi Kickflip/Kickflip70.avi Kickflip/Kickflip71.avi Kickflip/Kickflip72.avi Kickflip/Kickflip73.avi Kickflip/Kickflip74.avi Kickflip/Kickflip75.avi Kickflip/Kickflip76.avi Kickflip/Kickflip77.avi Kickflip/Kickflip78.avi Kickflip/Kickflip79.avi Kickflip/Kickflip80.avi Kickflip/Kickflip81.avi Kickflip/Kickflip82.avi Kickflip/Kickflip83.avi Kickflip/Kickflip84.avi Kickflip/Kickflip85.avi Kickflip/Kickflip86.avi Kickflip/Kickflip87.avi Kickflip/Kickflip88.avi Kickflip/Kickflip89.avi Kickflip/Kickflip90.avi Kickflip/Kickflip91.avi Kickflip/Kickflip92.avi Kickflip/Kickflip93.avi Kickflip/Kickflip94.avi Kickflip/Kickflip95.avi Kickflip/Kickflip96.avi Kickflip/Kickflip97.avi Kickflip/Kickflip98.avi Kickflip/Kickflip99.avi Kickflip/Kickflip100.avi Kickflip/Kickflip101.avi Kickflip/Kickflip102.avi Kickflip/Kickflip103.avi Kickflip/Kickflip104.avi Kickflip/Kickflip105.avi Kickflip/Kickflip106.avi Kickflip/Kickflip107.avi Kickflip/Kickflip108.avi Kickflip/Kickflip109.avi Kickflip/Kickflip110.avi Kickflip/Kickflip111.avi Kickflip/Kickflip112.avi Kickflip/Kickflip113.avi Ollie/Ollie71.avi Ollie/Ollie72.avi Ollie/Ollie73.avi Ollie/Ollie74.avi Ollie/Ollie75.avi Ollie/Ollie76.avi Ollie/Ollie77.avi Ollie/Ollie78.avi Ollie/Ollie79.avi Ollie/Ollie80.avi Ollie/Ollie81.avi Ollie/Ollie82.avi Ollie/Ollie83.avi Ollie/Ollie84.avi Ollie/Ollie85.avi Ollie/Ollie86.avi Ollie/Ollie87.avi Ollie/Ollie88.avi Ollie/Ollie89.avi Ollie/Ollie90.avi Ollie/Ollie91.avi Ollie/Ollie92.avi Ollie/Ollie93.avi Ollie/Ollie94.avi Ollie/Ollie95.avi Ollie/Ollie96.avi Ollie/Ollie98.avi Ollie/Ollie99.avi Ollie/Ollie100.avi Ollie/Ollie101.avi Ollie/Ollie102.avi Ollie/Ollie103.avi Ollie/Ollie104.avi Ollie/Ollie105.avi Ollie/Ollie106.avi Ollie/Ollie107.avi Ollie/Ollie0.avi Ollie/Ollie1.avi Ollie/Ollie2.avi Ollie/Ollie3.avi Ollie/Ollie4.avi Ollie/Ollie5.avi Ollie/Ollie6.avi Ollie/Ollie7.avi Ollie/Ollie8.avi Ollie/Ollie9.avi Ollie/Ollie10.avi Ollie/Ollie11.avi Ollie/Ollie12.avi Ollie/Ollie13.avi Ollie/Ollie14.avi Ollie/Ollie15.avi Ollie/Ollie16.avi Ollie/Ollie17.avi Ollie/Ollie18.avi Ollie/Ollie19.avi Ollie/Ollie20.avi Ollie/Ollie21.avi Ollie/Ollie22.avi Ollie/Ollie23.avi Ollie/Ollie24.avi Ollie/Ollie25.avi Ollie/Ollie26.avi Ollie/Ollie27.avi Ollie/Ollie28.avi Ollie/Ollie29.avi Ollie/Ollie30.avi Ollie/Ollie31.avi Ollie/Ollie32.avi Ollie/Ollie33.avi Ollie/Ollie34.avi Ollie/Ollie35.avi Ollie/Ollie36.avi Ollie/Ollie37.avi Ollie/Ollie38.avi Ollie/Ollie39.avi Ollie/Ollie40.avi Ollie/Ollie41.avi Ollie/Ollie42.avi Ollie/Ollie43.avi Ollie/Ollie44.avi Ollie/Ollie45.avi Ollie/Ollie46.avi Ollie/Ollie47.avi Ollie/Ollie48.avi Ollie/Ollie49.avi Ollie/Ollie50.avi Ollie/Ollie51.avi Ollie/Ollie52.avi Ollie/Ollie53.avi Ollie/Ollie54.avi Ollie/Ollie55.avi Ollie/Ollie56.avi Ollie/Ollie57.avi Ollie/Ollie58.avi Ollie/Ollie59.avi Ollie/Ollie60.avi Ollie/Ollie61.avi Ollie/Ollie62.avi Ollie/Ollie63.avi Ollie/Ollie64.avi Ollie/Ollie65.avi Ollie/Ollie66.avi Ollie/Ollie67.avi Ollie/Ollie68.avi Ollie/Ollie69.avi Ollie/Ollie70.avi'
all = all.split(' ')
# split the list into 2 lists depending on kickflip or ollie
kickflip = []
ollie = []
for i in all:
    if 'Kickflip' in i:
        kickflip.append(i)
    elif 'Ollie' in i:
        ollie.append(i)
# randomly suffle both lists
import random
random.shuffle(kickflip)
random.shuffle(ollie)

# print all items in kickflip
for i in kickflip:
    print(i)

# print all items in ollie
for i in ollie:
    print(i)

Kickflip/Kickflip111.avi
Kickflip/Kickflip95.avi
Kickflip/Kickflip51.avi
Kickflip/Kickflip20.avi
Kickflip/Kickflip80.avi
Kickflip/Kickflip70.avi
Kickflip/Kickflip104.avi
Kickflip/Kickflip65.avi
Kickflip/Kickflip32.avi
Kickflip/Kickflip64.avi
Kickflip/Kickflip87.avi
Kickflip/Kickflip54.avi
Kickflip/Kickflip45.avi
Kickflip/Kickflip60.avi
Kickflip/Kickflip23.avi
Kickflip/Kickflip63.avi
Kickflip/Kickflip74.avi
Kickflip/Kickflip15.avi
Kickflip/Kickflip47.avi
Kickflip/Kickflip17.avi
Kickflip/Kickflip42.avi
Kickflip/Kickflip76.avi
Kickflip/Kickflip8.avi
Kickflip/Kickflip88.avi
Kickflip/Kickflip99.avi
Kickflip/Kickflip25.avi
Kickflip/Kickflip110.avi
Kickflip/Kickflip53.avi
Kickflip/Kickflip21.avi
Kickflip/Kickflip30.avi
Kickflip/Kickflip22.avi
Kickflip/Kickflip69.avi
Kickflip/Kickflip33.avi
Kickflip/Kickflip100.avi
Kickflip/Kickflip58.avi
Kickflip/Kickflip68.avi
Kickflip/Kickflip40.avi
Kickflip/Kickflip109.avi
Kickflip/Kickflip85.avi
Kickflip/Kickflip14.avi
Kickflip/Kickflip26.avi
Kickflip/Kic

In [97]:
# Pipeline

import numpy as np
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture as GMM
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

pca_range = [64]
gnn_range = [3]
degrees = [1]
random_seeds = [1]

# Assuming train_videos and test_videos are already defined

for pca_n_components in pca_range:
    # PCA Transformation
    pca = PCA(n_components=pca_n_components)
    all_train_descriptors = np.concatenate([v.descriptors for v in train_videos], axis=0)
    pca.fit(all_train_descriptors)

    for v in train_videos:
        v.pca_descriptors = pca.transform(v.descriptors)

    for gmm_n_components in gnn_range:
        for random_seed in random_seeds:
            # GMM Model
            gmm = GMM(n_components=gmm_n_components, covariance_type='diag', random_state=random_seed)
            all_pca_descriptors = np.concatenate([v.pca_descriptors for v in train_videos], axis=0)
            gmm.fit(all_pca_descriptors)

            # Fisher Vectors for Training Data
            for v in train_videos:
                v.fisher_vector = fisher_from_descriptors(v.pca_descriptors, gmm)

            # Fisher Vectors for Test Data
            for v in test_videos:
                v.pca_descriptors = pca.transform(v.descriptors)
                v.fisher_vector = fisher_from_descriptors(v.pca_descriptors, gmm)

            # Define the hyperparameters to test
            param_grid = {'degree': degrees}

            # Create an SVM classifier
            svc = SVC(kernel='poly', probability=True)

            # Create a GridSearchCV object with the SVM classifier and the hyperparameters to test
            model = GridSearchCV(svc, param_grid, cv=5)

            # Fit the GridSearchCV object to the training set
            model.fit(X=[v.fisher_vector for v in train_videos], y=[v.label for v in train_videos])

            # Print the best hyperparameters and the corresponding accuracy score on the validation set
            print(f'Best hyperparameters: {model.best_params_}')
            print(f'Validation accuracy: {model.best_score_:.3f}')

            # Evaluate Classifier
            accuracy = model.score(X=[v.fisher_vector for v in test_videos], y=[v.label for v in test_videos])

            # Check training accuracy
            training_accuracy = model.score(X=[v.fisher_vector for v in train_videos], y=[v.label for v in train_videos])

            # Print Results
            print(f'PCA Components: {pca_n_components}, GMM Components: {gmm_n_components}, Accuracy: {accuracy}, gmm score: {gmm.score(all_pca_descriptors)}, training accuracy: {training_accuracy}, random seed: {random_seed}')

Best hyperparameters: {'degree': 1}
Validation accuracy: 0.644
PCA Components: 64, GMM Components: 3, Accuracy: 0.7142857142857143, gmm score: 41.36216987232699, training accuracy: 0.7006369426751592, random seed: 1


NameError: name 'filename' is not defined